In [5]:
import pandas as pd
import os
import json
from pandas.io.json import json_normalize
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

print(os.listdir())

['.circleci', '.git', '.github', '.gitignore', '.ipynb_checkpoints', 'askreddit_data.json', 'Bert ADL Project.ipynb', 'docker', 'docs', 'examples', 'hubconf.py', 'hubconfs', 'LICENSE', 'MANIFEST.in', 'notebooks', 'pytorch_pretrained_bert', 'README.md', 'requirements.txt', 'samples', 'setup.py', 'tests', 'uncased_askreddit_data.tsv']


In [2]:
data = json.load(open('./askreddit_data.json', 'r'))

# ask_reddit_data =  pd.read_json('./askreddit_data.json')
# ask_reddit_data = json_normalize(data)
uncased_ask_reddit_data = pd.read_csv('./uncased_askreddit_data.tsv', sep='\t', header=0)

In [3]:
# ask_reddit_data.head()#['comments'][0]

In [4]:
uncased_ask_reddit_data.head(20)

,submission score,comment score,submission id,comment id,title (string #1),comment (string#2)
0,80513,25495,bn3bpz,en27b8v,would you support a mandatory environmental se...,"not mandatory, but there was a voluntary progr..."
1,80513,4686,bn3bpz,en2anug,would you support a mandatory environmental se...,i feel like instead of forcing people to do so...
2,80513,2041,bn3bpz,en254lj,would you support a mandatory environmental se...,i wouldn't make it mandatory but maybe optiona...
3,80513,608,bn3bpz,en2gacy,would you support a mandatory environmental se...,"we already have americorps, where you serve fo..."
4,80513,310,bn3bpz,en2p5ro,would you support a mandatory environmental se...,i feel like forcing people to do it will just ...
5,80513,2580,bn3bpz,en22x9q,would you support a mandatory environmental se...,mandatory servitude is never a step in the rig...
6,80513,9453,bn3bpz,en21ky7,would you support a mandatory environmental se...,i don't support involuntary anything
7,80513,5045,bn3bpz,en22jzh,would you support a mandatory environmental se...,that sounds like slavery with extra steps
8,80513,141,bn3bpz,en32fz3,would you support a mandatory environmental se...,this is the most reddit thing i've read in a l...
9,80513,18,bn3bpz,en4942i,would you support a mandatory environmental se...,i oppose any form of conscription.


In [6]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to C:\Users\pedalo\AppData\Local\Temp\tmprwtohdv4
100%|███████████████████████████████████████████████████████████████████████| 231508/231508 [00:00<00:00, 625696.28B/s]
INFO:pytorch_pretrained_bert.file_utils:copying C:\Users\pedalo\AppData\Local\Temp\tmprwtohdv4 to cache at C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:removing temp file C:\Users\pedalo\AppData\Local\Temp\tmprwtohdv4
INFO:pytorch_pretrained_bert.tok

In [9]:
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']


In [10]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

In [ ]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
print(tokens_tensor)
print(segments_tensors)

In [16]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
if torch.cuda.is_available() == True:
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\pedalo\AppData\Local\Temp\tmpyib2v119
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":

In [21]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
if torch.cuda.is_available() == True:
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)
    print(predictions.size())
# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'
print(predicted_index)
print(predicted_token)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\pedalo\AppData\Local\Temp\tmpihu8lrez
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":

torch.Size([1, 14, 30522])
27227
henson
